In [1]:
# Human specific
import inspect

environment_name = "MultiAgentBlocksWorld"
specific = inspect.cleandoc("""\
                Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. 
                Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, and only interact with one block at a time. 
                Additionally, one agent can only interact with vowel blocks, and the other can only interact with consonant blocks. 
                Initially there are blocks A, B, C, O. A is on the table. B is on top of A, and C and O are on the table. 
                The goal is to have A on the table, B on the table, C on top of O, and O on top of B.
                """)  # This prompt is ignored if the environment_name already corresponds to an existing environment.
format = "json"

In [2]:
# Generate the json representation of the environment
from pathlib import Path

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT
from src.llm_plan.config import ENVIRONMENTS_JSON_PATH

model = ChatGPT("gpt-5-mini")  # This is one of the many models you can use now
planner = Planner()

plan_path = Path(f"{environment_name}.{format}")
full_path = ENVIRONMENTS_JSON_PATH / plan_path

# Skip if the plan already exists
if not full_path.exists():
    planner.generate_representation(model, specific, environment_name, format=format)
else:
    print(f"{full_path} already exists. Skipping generation.")
    print("[Warning]: The prompt `specific` will be ignored!")

environments/static/MultiAgentBlocksWorld.json already exists. Skipping generation.
[Warning]: The prompt `specific` will be ignored!


In [3]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
print("Plan:\n", env.plan)

Plan:
 [['vowel_agent.pddl', 'consonant_agent.pddl'], ['orchestrator.pddl']]


In [4]:
# Plan
model = ChatGPT("gpt-4o")  # switch to a less powerful model for planning
responses = planner.plan(model, env)

----- vowel_agent->pddl -----

----- System Prompt -----
You are an expert with PDDL problems Planning Domain Definition Language. You always provide a PDDL domain and a PDDL problem file to solve the task. You always enclose the pddl domain between <domain></domain> tags and the pddl problem between <problem></problem> tags.

----- Prompt -----
Your name is vowel_agent. You are in an environment with the following public information ['There are blocks labeled A B C O', 'Blocks are arranged in stacks on a table', 'Agents can only interact with the top block of each stack', 'Agents can only move one block at a time', 'One agent can only manipulate vowel labeled blocks and the other can only manipulate consonant labeled blocks', 'The target configuration is A on the table B on the table C on top of O and O on top of B'] You have the following knowledge ['I can only manipulate vowel blocks', 'I can move only one block at a time', 'I can only interact with the top block of a stack'] This i

In [5]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_scheduler" etc.
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [6]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [7]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)
    
with open(BASE_FOLDER / "logs.txt", "r") as f:
    pddl_logs = f.read()

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: ?X does not name a known token



In [15]:
# Hypervisor
from src.llm_plan.hypervisor import Hypervisor

prompt_args_hypervisor = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "syntax_errors": pddl_error,
    "pddl_logs": pddl_logs
}

hypervisor = Hypervisor(prompt_args_hypervisor)
response = hypervisor.run(model)

Attempting to load module from: /Users/emalfa/Desktop/MultiAgentPlanning/src/llm_plan/agent.py
Module agent loaded successfully!


In [19]:
# The class can be instantiated in this way
print(response)
agent_name = response.split("<class>")[-1].split("</class>")[0]
agent_class = hypervisor.agents[agent_name]
new_agent = agent_class(model, prompt_args_hypervisor)

# At each iteration, we can just update the prompt_args_hypervisor args and re-run the hypervisor

<class>AgentSyntaxPDDL</class>


In [ ]:
# Let's try to refine the plan syntax
from src.llm_plan.agent import (
    AgentDeepThinkPDDL,
    AgentSyntaxPDDL,
    AgentFastDownwardAdapter
    )

domain_name = "refined_domain_{i}.pddl"
problem_name = "refined_problem_{i}.pddl"
sas_plan_name = "refined_sas_plan_{i}"
logs_name = "refined_logs_{i}.txt"

num_attempts = 10
last_valid_plan = None

for i in range(num_attempts):
    
    # --- General validator of PDDL plans ---
    prompt_args_deepthink = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = AgentDeepThinkPDDL(llm=model, prompt_args=prompt_args_deepthink)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # --- Fast Downward adapter ---
    prompt_args_ff_adapter = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }
    
    hypervisor_fd = AgentFastDownwardAdapter(llm=model, prompt_args=prompt_args_ff_adapter)
    
    r = hypervisor_fd.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / problem_name.format(i=i), "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / domain_name.format(i=i), "w") as f:
        f.write(domain)
        
    # --- Launch the solver ---
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / sas_plan_name.format(i=i),
        BASE_FOLDER / domain_name.format(i=i),
        BASE_FOLDER / problem_name.format(i=i),
    ]

    with open(BASE_FOLDER / logs_name.format(i=i), "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / domain_name.format(i=i)} \
    {BASE_FOLDER / problem_name.format(i=i)} \
    {BASE_FOLDER / sas_plan_name.format(i=i)}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Store the last valid plan
    if (BASE_FOLDER / sas_plan_name.format(i=i)).exists():
        last_valid_plan = i

    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # --- Syntax validator ---
    print(f"Validation attempt {i+1}/{num_attempts}")
    
    with open(BASE_FOLDER / logs_name.format(i=i), "r") as f:
        pddl_logs = f.read()
        
    prompt_args_syntax_pddl = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
        "pddl_logs": pddl_logs
    }

    hypervisor_pddl = AgentSyntaxPDDL(llm=model, prompt_args=prompt_args_syntax_pddl)

    r = hypervisor_pddl.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

In [ ]:
# Final plan run and validation
assert last_valid_plan is not None, "No valid plan was found with refinements."

with open(BASE_FOLDER / problem_name.format(i=last_valid_plan), "w") as f:
    f.write(problem)

with open(BASE_FOLDER / domain_name.format(i=last_valid_plan), "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / sas_plan_name.format(i=last_valid_plan),
    BASE_FOLDER / domain_name.format(i=last_valid_plan),
    BASE_FOLDER / problem_name.format(i=last_valid_plan),
]

with open(BASE_FOLDER / logs_name.format(i=i), "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / domain_name.format(i=last_valid_plan)} \
{BASE_FOLDER / problem_name.format(i=last_valid_plan)} \
{BASE_FOLDER / sas_plan_name.format(i=last_valid_plan)}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

In [ ]:
# Report the natural language plan
from src.llm_plan.agent import AgentNaturalLanguage

with open(BASE_FOLDER / problem_name.format(i=last_valid_plan), "r") as f:
    problem = f.read()

with open(BASE_FOLDER / domain_name.format(i=last_valid_plan), "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / sas_plan_name.format(i=last_valid_plan), "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = AgentNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)